# Contents of Notebook: 

[Link to Initial data paths](#datapaths)
<a href='#datapaths'></a>

[Link to data processing functions](#dataprocessing)
<a href='#dataprocessing'></a>

[Metric Functions used for classification](#metricfunctions)
<a href='#metricfunctions'></a>

#### TLDR: 
In the initial_model notebook, we looked at the different applied models to the ABC + neural network embeddings dataset. This is a continuation notebook that focusses on data exploration

###### Existing datasets: 
[Regression dataset](#linearregdataset)<a href='#linearregdataset'></a> : 

Path: /mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/train_valid_test_all/x_train_reg_all.npz
Consists of : x_*_reg_all, y_*_reg_all, *_gene_labels_reg_all where * is train, valid, test respectively <br>
x_*_reg_all : <br>
sizes: Train (16525, 1000), Valid (3120, 1000), Test (1031, 1000) <br>
contains sequence embeddings <br>
y_*_reg_all : <br>
sizes: Train (16525, ), Valid (3120, ), Test (1031, ) : <br>
contains corresponding gene expression values in TPM 

*_gene_labels_reg_all: <br>
sizes : Train (16525, ), Valid (3120, ), Test (1031, ) : <br>
contains corresponding gene names 

<b>[Classification dataset]:</b> 

Path: /mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/train_classification_labels_abc_large.npz
consists of: *_classification_labels_abc_large where * is train, valid, test respectively <br>
*_classification_labels_abc_large : <br> 
sizes: Train (16525, 1000), Valid (3120, 1000), Test (1031, 1000) <br>
contains binary labels for gene expression for classification model <br>

NOTE: classification x inputs (embeddings) can use the ones for regression

[JUST expressed genes dataset](#expressedgenespath)<a href='#expressedgenespath'></a><br> 



### Contents:
1. [classification model](#classificationmodel)<a href='#classificationmodel'></a> 
2. [classification results](#classificationresults)<a href='#classificationresults'></a>
3. [Regression dataset used ](#linearregdataset)<a href='#linearregdataset'></a>
4. [Path to expressed genes dataset](#expressedgenespath)<a href='#expressedgenespath'></a><br> 
5. [Linear Regression w prior classification Results](#linregclass)<a href='#linregclass'></a>
6. [Linear Regressor w Boosting Algorithm ](#linboostingalgo)<a href='##linboostingalgo'></a><br>
6a. [Plot of predicted boosted value](#plotpredicted)<a href='#plotpredicted'></a><br>
6b. [Plot of Residual](#plotresidual)<a href='#plotresidual'></a><br>
6c. [Variance dsistribution](#variancedistribution)<a href='#variancedistribution'></a><br>
7. [GC Percentage correlation with neural network embeddings ](#gccorrelation)<a href='#gccorrelation'></a>
8. [Looking at data Distribution of Labels](#datadistribution)<a href='#datadistribution'></a>
9. [Threshold at 0.5](#threshold0.5)<a href='#threshold0.5'></a> 
10. [Threshold at 1.0](#threshold1.0)<a href='#threshold1.0'></a>
11. [GC content of the expressed genes and comparing the difference](#gc_interceptande)<a href='#gc_interceptande'></a><br>
11a. [Histogram plot of GC percent of neural network embeddings](#histoplot)<a href='#histoplot'></a><br>
11b. [Plot of histogram magnitude against GC content](#embedmag)<a href='#embedmag'></a><br>
11c. [Histogram plot of embedding distribution](#embeddist)<a href='#embeddist'></a><br>

In this notebook, I look at understanding why the linear model isn't fitting the model as well as hoped. 

I built a [classification model](#classificationmodel)<a href='#classificationmodel'></a> with the corresponding [results](#classificationresults)<a href='#classificationresults'></a> <br> 
The classification model was able to achieve an accuracy of ~0.7 at predicting whether the gene was on or off based on its weighted embeddings

Utilizing the [Regression dataset](#linearregdataset)<a href='#linearregdataset'></a> , 
I looked at fitting a Linear Regression after prior classification. The linear regression model was trained on ONLY expressed genes. <br> 

[Path to expressed genes dataset](#expressedgenespath)<a href='#expressedgenespath'></a><br> 

Linear Regression after classification got rid of the previous vertical line that was previously present in correlation graphs in the initial model notebook. 
[Results](#linregclass)<a href='#linregclass'></a> 

A boosting algorithm was applied where at each iteration, the linear regressor was fitted to the residual of y_actual - y_hat. This was done to allow multiple iterations of fitting in the hopes that the final residual would be the noise. 
[Linear Regressor Boosting](#linboostingalgo)<a href='##linboostingalgo'></a> 

The "predicted" value was plotted against true labels [plot](#plotpredicted)<a href='#plotpredicted'></a>
The "residual" value was plotted against true labels [plot](#plotresidual)<a href='#plotresidual'></a>

The expectation was high correlation of "Predicted values" with true labels and poor correlation of residual values with true labels. 

However, This was not the case .... 

Time to look at how the variance of the data is being distributed, w.r.t the above plots
[Variance dsistribution](#variancedistribution)<a href='#variancedistribution'></a>
   
Most of the variance of the data is captured in the residual which led us to the conclusion that a linear model could not fit this data well. 

Do the neural network embeddings correlate highly with GC content of sequence? Could this be the reason why linear regression does not work? 
[GC Correlation](#gccorrelation)<a href='#gccorrelation'></a>

At first glance, you could infer slight correlation. However, I need to do more digging into this. 
   
[Looking at data Distribution of labels](#datadistribution)<a href='#datadistribution'></a> made us realize that fitting a two-part linear model such as a zero-inflated negative binomal or a generalized linear model negative binomial might work better [This is on TODO]

Another thing to try was also to artificially Thresholding data distribution of labels: 
1. [Threshold at 0.5](#threshold0.5)<a href='#threshold0.5'></a> 
2. [Threshold at 1.0](#threshold1.0)<a href='#threshold1.0'></a>

Both these two linear models with the new artificially curated data distribution had an odd [intercept](#whatintercept)<a href='#whatintercept'></a> 

I looked into [GC content of the expressed genes and comparing the difference](#gc_interceptande)<a href='#gc_interceptande'></a> I looked into the GC content of x inputs of the expressed genes whose predicted value was along the intercept and NOT along the intercept
This analysis was done in 3 ways: 
1. Looking at a [histogram](#histoplot)<a href='#histoplot'></a> plot of these expressed genes a) at the intercept b) not at the intercept 
2. Looking at the [embedding magnitude](#embedmag)<a href='#embedmag'></a> against the percent GC content and seeing if there is a relationship 
3. Looking at the [embedding distribution](#embeddist)<a href='#embeddist'></a> 
The range of values that the input embeddings of the expressed genes a) at the intercept b) not at the intercept.

#### Import necessary libraries

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import pickle
import joblib
import sklearn
import scipy
from sklearn.ensemble import RandomForestRegressor
import pickle
import plotly.graph_objects as go

%matplotlib inline

<a id='datapaths'></a>
#### Data paths

In [ ]:
labels = np.load("/mnt/lab_data2/kmualim/enhancer_gene_anlysis/datasets/all_dataset/gene_labels_all.npy", allow_pickle=True)
express_1 = pickle.load(open("/mnt/lab_data2/kmualim/enhancer_gene_anlysis/datasets/all_dataset/embeddings_all_w_gene.p","rb"))
expression = pd.read_csv("/mnt/lab_data2/kmualim/enhancer_gene_anlysis/datasets/K562.Genes.TPM.txt", sep=" ", header=None)
expression.head()

<a id='dataprocessing'></a>
#### Functions for combining and splitting data inputs 

In [ ]:
## Appending labels to expression table 
## 0 for non-expressed, 0 for expressed
label_1=[]
for i in expression[1]: 
    if i>1: 
        label_1.append(1)
    else: 
        label_1.append(0)

expression[2] = label_1

In [ ]:
from sklearn.model_selection import train_test_split  

def get_expression(express_dict, y_valid_labels): 
    x_val=[]
    for i in y_valid_labels[0]:
        x_val.append(express_dict[i])
    x_values = np.vstack(x_val)
    return x_values
    
def get_list(y_valid):
    y_valid_el = []
    for i in y_valid: 
        y_valid_el.append(i)
    return y_valid_el

# creating feature spaces 
def get_train_valid_split(labels, expression_dict, expression, classify=False):
    y_train, y_rem = train_test_split(labels, test_size=0.2, random_state=42, shuffle=True)
    y_valid, y_test = train_test_split(y_rem, test_size=0.25, random_state=42, shuffle=True)
    y_train_el = get_list(y_train) 
    y_valid_el = get_list(y_valid) 
    y_test_el = get_list(y_test) 
    y_train_labels = expression[expression[0].isin(y_train_el)]
    y_valid_labels = expression[expression[0].isin(y_valid_el)]
    y_test_labels = expression[expression[0].isin(y_test_el)]
    x_train = get_expression(expression_dict, y_train_labels)
    x_valid = get_expression(expression_dict, y_valid_labels)
    x_test = get_expression(expression_dict, y_test_labels)
    #if classify: 
    y_train_c, y_valid_c, y_test_c = get_class_labels(y_train_labels, y_valid_labels, y_test_labels)
    #else: 
    y_train_r, y_valid_r, y_test_r = get_reg_labels(y_train_labels, y_valid_labels, y_test_labels)
    return x_train, x_valid, x_test, y_train_r, y_valid_r, y_test_r, y_train_c, y_valid_c, y_test_c

def get_class_labels(y_train_labels, y_valid_labels, y_test_labels):
        y_train = y_train_labels[2].values
        y_valid = y_valid_labels[2].values
        y_test = y_test_labels[2].values 
        return y_train, y_valid, y_test
    
def get_reg_labels(y_train_labels, y_valid_labels, y_test_labels):
        y_train = y_train_labels[1].values
        y_valid = y_valid_labels[1].values
        y_test = y_test_labels[1].values
        return y_train, y_valid, y_test
    
    
    
%matplotlib inline
from scipy.stats import spearmanr,pearsonr
import matplotlib.pyplot as plt

def plot(y_valid_preds, y_valid): 
    print("Spearman R", spearmanr(y_valid_preds,np.arcsinh(y_valid)))
    print("Pearson R", pearsonr(y_valid_preds,np.arcsinh(y_valid)))
    plt.plot(y_valid_preds, np.arcsinh(y_valid), 'ro', markersize=2)
    plt.xlabel('Predictions')
    plt.ylabel('Actual')
    plt.title('Correlation graph')

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - np.arcsinh(test_labels))
    plot(predictions, test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return errors #accuracy

<a id='datafiles'></a>
#### Data files and corresponding names for classification

In [ ]:
x_train, x_valid, x_test, y_train, y_valid, y_test, y_train_labels, y_valid_labels, y_test_labels = get_train_valid_split(labels, express_1, expression, classify=True)

In [ ]:
np.savez("train_classification_labels_abc_large.npz",train_preds)

In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

<a id='metricfunctions'></a>
#### metric functions

In [ ]:
# ACCURACY PROLLY NOT THE BEST METRIC FOR 2:1 CLASS IMBALANCE 
def get_accuracy(model, x_train, y_train):
    preds = model.predict(x_train)
    print("Accuracy: ", accuracy_score(y_train, preds))
    return preds

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
def plot_precision_recall_curve(y_preds, y_train): 
    precision, recall, _ = precision_recall_curve(y_preds, y_train)

<a id='classificationmodel'></a>
#### Building classification model 

Labeling genes that have TPM > 0 with a class label =1 , else class label = 0 

In [ ]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
## Figuring out class proportions
print("Number of labels in class 0: ",len(y_train[y_train==0]))
print("Number of labels in class 1: ",len(y_train[y_train==1]))


In [ ]:
# Logistic Regression CV, CV =10 ， accuracy is ~ 73%
log_reg = LogisticRegressionCV(cv=10).fit(x_train, y_train)

In [ ]:
joblib.dump(log_reg, open("logreg_abc_large.joblib", "wb"))

In [ ]:
log_reg=joblib.load("logreg_abc_large.joblib")

<a id='classificationresults'></a>
#### Predicted classifcation labels

In [ ]:
train_preds = get_accuracy(log_reg, x_train, y_train)

In [ ]:
valid_preds = get_accuracy(log_reg, x_valid, y_valid)

In [ ]:
test_preds = get_accuracy(log_reg, x_test, y_test)

In [ ]:
np.savez("train_classification_labels_abc_large.npz",train_preds)

In [ ]:
np.savez("valid_classification_labels_abc_large.npz",valid_preds)

In [ ]:
np.savez("test_classification_labels_abc_large.npz",test_preds)

In [ ]:
train_preds = np.load("train_classification_labels_abc_large.npz")['arr_0']

In [ ]:
valid_preds = np.load("valid_classification_labels_abc_large.npz")['arr_0']

In [ ]:
test_preds = np.load("test_classification_labels_abc_large.npz")['arr_0']

<a id='linearregdataset'></a>
#### path to Linear regression dataset

In [ ]:
x_train_reg, x_valid_reg, x_test_reg, y_train_reg, y_valid_reg, y_test_reg, y_train_labels, y_valid_labels, y_test_labels = get_train_valid_split(labels, express_1, expression, classify=False)


In [ ]:
# grabbing all training_data 
np.savez("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/train_valid_test_all/x_train_reg_all.npz", x_train_reg)
np.savez("datasets/train_valid_test_all/x_valid_reg_all.npz", x_valid_reg)
np.savez("datasets/train_valid_test_all/x_test_reg_all.npz", x_test_reg)
np.savez("datasets/train_valid_test_all/y_train_reg_all.npz", y_train_reg)
np.savez("datasets/train_valid_test_all/y_valid_reg_all.npz", y_valid_reg)
np.savez("datasets/train_valid_test_all/y_test_reg_all.npz", y_test_reg)
np.savez("datasets/train_valid_test_all/train_gene_labels_reg_all.npz", y_train_labels)
np.savez("datasets/train_valid_test_all/valid_gene_labels_reg_all.npz", y_valid_labels)
np.savez("datasets/train_valid_test_all/test_gene_labels_reg_all.npz", y_test_labels)


In [ ]:
def plot_after_class(y_valid_preds, y_valid): 
    print("Spearman R", spearmanr(y_valid_preds,np.arcsinh(y_valid)))
    print("Pearson R", pearsonr(y_valid_preds,np.arcsinh(y_valid)))
    plt.plot(y_valid_preds, np.arcsinh(y_valid), 'ro', markersize=2)
    plt.xlabel('Predictions')
    plt.ylabel('Actual')
    plt.title('Correlation graph')

def get_number_of_zeros(test_labels, preds): 
    num_test_zero=0
    num_pred_zero=0
    for i in range(len(test_labels)): 
        if test_labels[i]==0 and preds[i]!=0:
            num_test_zero+=1
        if test_labels[i]!=0 and preds[i]==0:
            num_pred_zero+=1
    return num_test_zero, num_pred_zero
    
def eval_after_class(model, test_features, test_labels, test_class_labels, include_zeros=True):
    predictions = model.predict(test_features)
    errors = abs((predictions*test_class_labels) - np.arcsinh(test_labels))
    final_preds = predictions*test_class_labels
    num_test_zero, num_pred_zero = get_number_of_zeros(test_labels, test_class_labels)
    if include_zeros:
        plot_after_class(final_preds, test_labels)
    else: 
        plot_after_class(final_preds[final_preds!=0], test_labels[final_preds!=0])
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    
    print("Number of Predictions!=0 and Actual==0:", num_test_zero)
    print("Number of Predictions==0 and Actual!=0:", num_pred_zero)
    print("Number of total Predictions:", len(final_preds))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return errors, predictions #accuracy

Grabbing only the labels that are 1 "expressed" 
use as input for linear regression 

Reduces training data from ~16k to just ~7k 

In [ ]:
x_train_rem = x_train_reg[train_preds!=0]
y_train_rem = y_train_reg[train_preds!=0]
x_valid_rem = x_valid_reg[valid_preds!=0]
y_valid_rem = y_valid_reg[valid_preds!=0]
x_test_rem = x_test_reg[test_preds!=0]
y_test_rem = y_test_reg[test_preds!=0]

In [ ]:
# train and train_labels are the concatenation of training and validation 
train = np.concatenate((x_train_reg, x_valid_reg))
train_labels = np.concatenate((y_train_reg, y_valid_reg))

# big_train_class_labels gives the classification labels for expressed and non-expressed genes 
big_train_class_labels = np.concatenate((train_preds, valid_preds))
train_big = train[big_train_class_labels!=0]
train_labels_big = train_labels[big_train_class_labels!=0]
print(len(train_big))

train_big and train_labels_big refer to x and y inputs for expressed_genes

<a id='expressedgenespath'></a>
#### paths to expressed genes datafile

In [ ]:
np.savez("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed.npz", train_big)
np.savez("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed_labels.npz", train_labels_big)

In [ ]:
train_big = np.load("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed.npz")['arr_0']
train_labels_big = np.load("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed_labels.npz")['arr_0']

In [ ]:
x_train_big, x_valid_big, y_train_big, y_valid_big = train_test_split(train_big, train_labels_big, test_size=0.2, random_state=42)

In [ ]:
print(len(train_big))

<a id='linregclass'></a>
#### Linear Regression after classification of genes

linreg_AFTERCLASS trained on ~ 7500 examples ("EXPRESSED")

In [ ]:
linreg_AFTERCLASS = LinearRegression()
linreg_AFTERCLASS.fit(train_big, np.arcsinh(train_labels_big))

In [ ]:
intercept=linreg_AFTERCLASS.intercept_

In [ ]:
linreg_AFTERCLASS.intercept_

Looking at how linear regression after classification applies to trainin set

In [ ]:
_, y_hat = eval_after_class(linreg_AFTERCLASS, x_train_reg, y_train_reg, train_preds)
plt.xlim(-1,10)

Looking at just the expressed portion of the above graph where vertical line at zero is omitted

In [ ]:
evaluate(linreg_AFTERCLASS, x_train_rem, y_train_rem)

Looking at how linear regression after classification applies to validation set

In [ ]:
_, y_valid_hat = eval_after_class(linreg_AFTERCLASS, x_valid_reg, y_valid_reg, valid_preds)
plt.xlim(-1,10)

Looking at just the expressed portion of the above graph where vertical line at zero is omitted

In [ ]:
evaluate(linreg_AFTERCLASS, x_valid_rem, np.arcsinh(y_valid_rem))
plt.xlim(0,10)

In [ ]:
y_hat_intercept = y_hat*train_preds
y_hat_v_intercept = y_valid_hat*valid_preds

In [ ]:
print(len(y_hat_intercept[y_hat_intercept==intercept]))

In [ ]:
print(len(y_hat_v_intercept[y_hat_v_intercept==intercept]))

<a id='linboostingalgo'></a>
#### Applying boosting algorithm for Linear regression algorithm for EXPRESSED GENES

In [ ]:
### Running iterations to minimize loss between actual and predictions 
linreg = LinearRegression()
y_next = np.arcsinh(train_labels_big)
scored_preds = 0
d = {}
for t in range(1,20): 
    linreg.fit(train_big, y_next)
    d[t]=linreg.coef_
    y_train_preds = linreg.predict(train_big)
    scored_preds += y_train_preds
    y_next = y_next - y_train_preds 
    

Plotting the magnitude of coefficients of the linear regression at each iteration
- it should *technically* be decreasing with each iteration 

In [ ]:
### fix the underflow overflow problem happeningg here 
coefficients=[]
for i in range(1,20):
    total = np.log((d[i]**2)+0.001
    coefficients.append(np.sum(total))
    

In [ ]:
x = np.linspace(1,20,19)
plt.plot(x, coefficients, 'ro')
plt.xlabel("Iteration Number")
plt.ylabel("Magnitude of coefficients")

cool...

In [ ]:
linreg_intercept = linreg.intercept_

<a id='plotpredicted'></a>
#### Plotting "predicted" signal against actual signal

In [ ]:
plot(scored_preds, train_labels_big)
plt.xlim(-10,15)

<a id='plotresidual'></a>
#### Plotting "residual" signal against actual signal

In [ ]:
plot(y_next, train_labels_big)
plt.xlim(-10,15)

In [ ]:
plt.plot(y_next, np.arcsinh(train_labels_big), 'ro')
plt.plot(scored_preds, np.arcsinh(train_labels_big), 'bo')
plt.xlim(-10,15)

<a id='variancedistribution'></a>
#### Understanding the variance from the residual signal relative to the "predicted" and actual signal 

In [ ]:
residual = (scored_preds - train_labels_big)**2

In [ ]:
predictions_1 = np.sum(scored_preds**2)

In [ ]:
actual = np.sum(train_labels_big**2)

In [ ]:
total = np.sum(residual)

In [ ]:
print("Variance of Predictions:", predictions_1)
print("Variance of Actual:", actual)
print("Variance of Residual:", total)

Are the neural network embeddings really being driven by 

<a id='gccorrelation'></a>
#### Neural network embeddings correlate strongly with GC content 

In [ ]:
GC_content_all = pd.read_csv("../scripts/data/cut_5_embeddings_all_GC.txt")

In [ ]:
def load_embedding(fname_npz):
    data=np.load(fname_npz,allow_pickle=True)
    regions=data['bed_entries']
    if 'embeddings' in data: 
        embeddings=data['embeddings']
        data_type='embedding'
    else:
        embeddings=data['deeplift_scores']
        data_type='deeplift_scores'
    return regions,embeddings,data_type

In [ ]:
_,get_embeddings,_ = load_embedding('../scripts/data/k562_dnase_allputative_classification_embeddings.0.-3.npz')

In [ ]:
all_sum_coef = [np.sum(i**2) for i in get_embeddings]

In [ ]:
GC_content_all['embeddings'] = all_sum_coef

In [ ]:
sort_GC_content_all = GC_content_all.sort_values(by=['5_pct_gc'])

In [ ]:
### Getting the mean and stdev of magnitude coefficient values depending on GC content 
sort_GC_content_all.groupby(sort_GC_content_all['5_pct_gc']).mean()
sort_GC_content_all.groupby(sort_GC_content_all['5_pct_gc']).std()
plt.plot(sort_GC_content_all['5_pct_gc'], sort_GC_content_all['embeddings'], 'ro')
plt.xlabel('gc percent')
plt.ylabel('embeddings magnitude')
plt.title('GC content v.s embeddings magnitude')

This is way too clumpy, let's try to make this better!

In [ ]:
sort_GC_content_all.to_csv("datasets/sorted_GC_content_all.csv", index=False)

In [ ]:
sort_GC_content_all= pd.read_csv("datasets/sorted_GC_content_all.csv", sep=",")

In [ ]:
dist = sort_GC_content_all.groupby(pd.cut(sort_GC_content_all['5_pct_gc'], np.arange(0,1.1, 0.1))).count()

In [ ]:
dist['embeddings']

In [ ]:
arr_a = [0, 0, 236100, 2564047, 6137750, 3628275, 1383575, 240066, 3759, 0]
x = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5]
fig = go.Figure(data=go.Scatter(x=x, y=arr_a))
fig.update_layout(title='Histogram of number of sequence embeddings based on percent GC content', 
                   xaxis_title='Percent of GC',
                   yaxis_title='Number of sequence embeddings')
fig.show()

#### Plotting histogram and Scatter plot of GC content v.s embeddings

In [ ]:
import plotly
import plotly.express as px
import plotly.offline as pyo
pyo.init_notebook_mode()
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
fig_scatter = go.Figure(data=go.Scatter(
    x=sort_GC_content_all['5_pct_gc'], 
    y=sort_GC_content_all['embeddings'],
    mode='markers',
    marker=dict(
        size=6, 
        color=sort_GC_content_all['5_pct_gc'],
        colorscale='Viridis', 
        showscale=True
               )
))

In [ ]:
fig_scatter.show()

GC content and embeddings distribution

In [ ]:
sort_GC_content_all.groupby(pd.cut(sort_GC_content_all['5_pct_gc'], np.arange(0,1.1, 0.1))).mean()

In [ ]:
sort_GC_content_all.groupby(pd.cut(sort_GC_content_all['5_pct_gc'], np.arange(0,1.1, 0.1))).std()

linreg_AFTERCLASS trained on ~6k examples

In [ ]:
joblib.dump(linreg_AFTERCLASS, open("linreg/linreg_abc_large_after_classification_TPM1.joblib", "wb"))

Getting y-intercept of linear regression

<a id='#datadistribution'></a>
#### Data distribution
Plotting the distribution of arcsinh(TPM) values , manually picking out only the values where TPM != 0 

In [ ]:
# a lot of the zeros are also from the arcsinh of the thing 
print(len(y_train_reg[y_train_reg==0]))
plt.hist(np.arcsinh(y_train_reg[y_train_reg!=0]), bins=100)
plt.show()

Y_TRAIN LABELS are np.arcsinh(TPM) <br>
Looking at the distribution, thresholding labels to just consider np.arcsinh(TPM) values above 0.5

In [ ]:
def plot_th(y_valid_preds, y_valid): 
    print("Spearman R", spearmanr(y_valid_preds,y_valid))
    print("Pearson R", pearsonr(y_valid_preds,y_valid))
    plt.plot(y_valid_preds, y_valid, 'ro', markersize=2)
    plt.xlabel('Predictions')
    plt.ylabel('Actual')
    plt.title('Correlation graph')

def eval_th(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    plot_th(predictions, test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return errors #accuracy

In [ ]:
y_val = np.arcsinh(y_train_reg)
rel = y_val[y_val>0.5]
x_train_rel = x_train_reg[y_val>0.5]

In [ ]:
y_val_1 = np.arcsinh(y_valid_reg)
y_val_rel = y_val_1[y_val_1>0.5]
x_valid_rel = x_valid_reg[y_val_1>0.5]

In [ ]:
print(len(rel))
print(len(y_val_rel))

<a id='threshold0.5'></a>
#### Thresholding at np.arcsinh(TPM) = 0.5

Data distribution at np.arcsinh(TPM) threshold = 0.5

In [ ]:
plt.hist(rel, bins=100)
plt.show()

Fitting linear regression using only TPM y values above a threshold of 0.5

In [ ]:
linreg_th = LinearRegression()
linreg_th.fit(x_train_rel, rel)

In [ ]:
linreg_th.intercept_

Correlation graph fit with train predicted and train actual labels

In [ ]:
eval_th(linreg_th, x_train_rel, rel)

Correlation graph fit with validation predicted and valid actual labels

In [ ]:
evaluate(linreg_th, x_valid_rel, y_val_rel)
plt.xlim(0,10)

In [ ]:
joblib.dump(linreg_th, open("linreg/linreg_threshold_0.5_abc_large.joblib", "wb"))

<a id='threshold1.0'></a>
#### Thresholding at np.arcsinh(TPM) = 1.0

In [ ]:
linreg_AFTERCLASS = joblib.load("linreg/linreg_abc_large_after_classification_TPM1.joblib")
linreg_AFTERCLASS.intercept_

Y_TRAIN LABELS are np.arcsinh(TPM) <br>
Thresholding labels to just consider np.arcsinh(TPM) values above 1.0

In [ ]:
y_val = np.arcsinh(y_train_reg)
rel = y_val[y_val>1.0]
x_train_rel = x_train_reg[y_val>1.0]

y_val_1 = np.arcsinh(y_valid_reg)
y_val_rel = y_val_1[y_val_1>1.0]
x_valid_rel = x_valid_reg[y_val_1>1.0]

Distribution of data when thresholded at 1.0

In [ ]:
plt.hist(rel, bins=100)
plt.show()

Fitting linear regression using only TPM y values above a threshold of 1.0 

In [ ]:
linreg_th_1 = LinearRegression()
linreg_th_1.fit(x_train_rel, rel)

In [ ]:
linreg_th_1.intercept_

Correlation graph fit with train predicted and train actual labels

In [ ]:
eval_th(linreg_th_1, x_train_rel, rel)

Correlation graph fit with valid predicted and valid actual labels

In [ ]:
evaluate(linreg_th_1, x_valid_rel, y_val_rel)
plt.xlim(0,6)

In [ ]:
joblib.dump(linreg_th_1, open("linreg/linreg_threshold_1_abc_large.joblib", "wb"))

<a id='whatintercept'></a>
### Investigating mass of same values for genes/data points

In [ ]:
linreg = joblib.load("/mnt/lab_data2/kmualim/Jamboree_data/models/abc_all/test_models/ABC.Score_linreg.joblib")

CORRELATION GRAPH ON TRAIN

In [ ]:
## y_intercept 
cept=linreg.intercept_

In [ ]:
preds = linreg.predict(x_train)
valid_preds = linreg.predict(x_valid)
test_preds = linreg.predict(x_test)

In [ ]:
print(len(preds))
print(len(preds[preds!=cept]))
print(len(preds[preds==cept]))
## 4% of total predictions 

In [ ]:
print(len(valid_preds))
print(len(valid_preds[valid_preds==cept]))

X_TRAIN_TI is the x_train input of predictions of expressed genes that are along the intercept <br>
X_TRAIN_E is the x_train input of the predictions of expressed genes that aren't along the intercept 

train_TI
valid_TI
test_TI 
are train, valid, test values with predicted values == intercept 

In [ ]:
## Labels to investigate
x_train_TI= x_train[preds==cept]
x_train_E = x_train[preds!=cept]
y_train_E = y_train_labels[preds!=cept]
#x_valid_T1=
#x_test_T1=
y_train_TI = y_train_labels[preds==cept]
y_valid_TI = y_valid_labels[valid_preds==cept]
y_test_TI = y_test_labels[test_preds==cept]


In [ ]:
np.savez("genes_expressed_genes_and_intercept.npz", y_train_TI)
np.savez("genes_expressed_genes_and_not_intercept.npz", y_train_E)

<a id='#gc_interceptande'></a>
###  Looking at the GC content of y_train_TI and y_train_E and comparing the difference

In [ ]:
genelist = pd.read_csv("/mnt/lab_data2/kmualim/Jamboree_data/GeneList.TSS1kb.GC.bed", sep="\t")

In [ ]:
genelist.head()

In [ ]:
## Gets the GC content of gene promoters 
genes_tss = genelist['4_usercol']
print(genes_tss[0])
genes_tss_GC = genelist['8_pct_gc']
print(genes_tss_GC[0])
genes_GC = {}

for i in range(len(genes_tss)):
    genes_GC[genes_tss[i]]= genes_tss_GC[i]

In [ ]:
### LOOKING AT JUST EXPRESSED GENES
y_train_TI_EXPRESSED = y_train_TI[y_train_TI[2]!=0]
y_train_E_EXPRESSED = y_train_E[y_train_E[2]!=0]

In [ ]:
x_train_of_expressed_genes_sq = [np.sum(i**2) for i in x_train_of_expressed_genes]
x_genes_E = [np.sum(i**2) for i in x_train_E]

In [ ]:
print("List of train genes that contains expression values == intercept :", len(train_TI))
print("List of non-expressed genes :", len(train_TI[train_TI[2]==0]))
print("List of valid genes that contains expression values == intercept :",len(valid_TI))
print("List of test genes that contains expression values == intercept :",len(test_TI))

In [ ]:
# Are these genes ubiquitously expressed genes ? 
ubi_e_genes = pd.read_csv("/users/kmualim/updated_ABC/ABC-Enhancer-Gene-Prediction/example/config/UbiquitouslyExpressedGenesHG19.txt", header=None)
ubi_e_genes.head()
expressed_genes = train_TI[train_TI[2]!=0]


x_train_of_expressed_genes = x_train_TI[train_TI[2]!=0]
y_train_of_expressed_genes = y_train_TI[train_TI[2]!=0]

In [ ]:
genes_TI_val = [genes_GC[i] for i in y_train_of_expressed_genes[0]]
genes_E_val = [genes_GC[i] for i in y_train_E[0]]

<a id='histoplot'></a>
#### Histogram plots of GC content distribution between expressed genes whose predicted value is AT intercept (Fig1)
#### and NOT at intercept (Fig 2) 

Looking at the GC content of expressed genes at are NOT the INTERCEPT 

In [ ]:
plt.hist(genes_E_val, bins=10)

Looking at the GC content distribution of expressed genes at are at the INTERCEPT 

In [ ]:
plt.hist(genes_TI_val, bins=10)

<a id='embedmag'></a>
#### looking at embeddings magnitude distribution of expressed genes whose predicted value is NOT AT intercept (Fig1) and AT intercept (Fig 2)

In [ ]:
plt.plot(x_genes_E, genes_E_val, 'bo')
plt.xlabel("Magnitude of coefficients")
plt.ylabel("Percent of GC content")
plt.title("Plot of expressed genes and genes NOT at intercept")

In [ ]:
plt.plot(x_train_of_expressed_genes_sq, genes_TI_val, 'ro')
plt.xlabel("Magnitude of coefficients")
plt.ylabel("Percent of GC content")
plt.title("Plot of expressed genes and genes at intercept")

<a id='embeddist'></a>
#### Distribution of embedding values plotted in histogram

Distribution of x_input of expressed and intercept predicted value genes 

In [ ]:
# Distribution of x_input of expressed and intercept predicted value genes
#print(len(x_train_of_expressed_genes))
plt.hist(x_train_of_expressed_genes[1], bins=10, label=1)
plt.hist(x_train_of_expressed_genes[73], bins=10, label=73)
#plt.hist(x_train_of_expressed_genes[91], bins=10, label=91)
#plt.hist(x_train_of_expressed_genes[145], bins=10, label=145)
#plt.hist(x_train_of_expressed_genes[163], bins=10, label=163)
plt.legend(loc="upper right")
plt.show(0.02)

In [ ]:
distribution=[]
for i in range(len(x_train_of_expressed_genes)):
    distribution.append(np.sum(x_train_of_expressed_genes[i])**2)

Looking at the sum of the squared distribution of coefficients in the x_input of expressed and intercept predicted value genes

In [ ]:
plt.hist(distribution, bins=10)
plt.show()
print("Mean distribution",np.mean(distribution))
print("Mean stddev", np.std(distribution))

Distribution of values in x_input of expressed and non-intercept predicted value genes 

In [ ]:
# Distribution of values in x_input of expressed and non-intercept predicted value genes 
plt.hist(x_train_E[5], bins=10, label=5)
plt.hist(x_train_E[10000], bins=10, label=10000)
plt.legend(loc="upper right")
plt.show(0.02)

In [ ]:
distribution_b=[]
for i in range(len(x_train_E)):
    distribution_b.append(np.sum(x_train_E[i])**2)

Looking at the sum of the squared distribution of coefficients in the x_input of expressed and non-intercept predicted value genes

In [ ]:
plt.hist(distribution_b, bins=10)
plt.show()
print("Mean distribution",np.mean(distribution_b))
print("Mean stddev", np.std(distribution_b))

In [ ]:
# Are any of these expressed genes ubiquitously expressed genes?? 
print(len(expressed_genes[expressed_genes[0].isin(ubi_e_genes)]))